Start a selenium http express server?

This convert webdriver commands to chrome commands just like chromedriver but implemented in JavaScript directly.



In [ ]:
var importer = require('../Core')
var fs = require('fs');
var path = require('path');
var bodyParser = require('body-parser')
var io = require('socket.io-client');
var selenium = require('express')();
var seleniumServer = require('http').createServer(selenium);
var client = io('https://localhost:8000', {
    secure: true,
    rejectUnauthorized: false 
});

var TOKEN_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.credentials');
var SESSIONS_PATH = path.join(TOKEN_DIR, 'sessions.txt');
var sessionId = fs.readFileSync(SESSIONS_PATH).toString().split('\n')[0];

var promisifyChrome = (...args) => {
    return new Promise((resolve, reject) => {
        var subscriber;
        subscriber = client.on('result', function (name, err, data) {
            // TODO: not just verify the service,
            //   also verify the tab id matches
            if(name === 'BrowserService.prototype.chrome') {
                subscriber.off();
                if(err !== null) {
                    return reject(err);
                }
                return resolve.apply(this, JSON.parse(data));
            }
        });
        client.emit.apply(client, [
            'call',
            'BrowserService.prototype.chrome',
            ...args
        ]);
    });
};

selenium.use(bodyParser.json());       // to support JSON-encoded bodies
selenium.use(bodyParser.urlencoded({     // to support URL-encoded bodies
  extended: true
}));
selenium.all('/wd/hub/*', function(req, res, next) {
    console.log(req.originalUrl);
    var tabId;
    if(req.originalUrl.match(/url$/ig)) {
        console.log(req.body)
        promisifyChrome('windows.getAll', {})
        .then(windows => importer.runAllPromises(windows
            .map(w => resolve => 
                 promisifyChrome('tabs.getAllInWindow', w.id)
                .then(tabs => {
                    for (const t of tabs) {
                        console.log(t.url + ' - ' + req.body.url)
                        if (t.url.indexOf(req.body.url) > -1) {
                            tabId = t.id;
                            return promisifyChrome(
                                'tabs.update', tabId, {active: true});
                        }
                    }
                })
                .then(r => resolve()))))
        .then(() => promisifyChrome('debugger.attach', {
            tabId
        }, '1.1'))
        .then(() => promisifyChrome('debugger.sendCommand', {
            tabId
        }, 'Page.navigate', {
            url: req.body.url}))
        .then(() => {
            res.setHeader('Content-Type', 'application/json');
            res.send(JSON.stringify({
                value: {}
            }));
        })
        .catch(e => console.log(e))
        /*
        client.emit('call',
                    'BrowserService.prototype.chrome',
                    'tabs.getAllInWindow',
                    {})
        
        client.emit('call',
                    'BrowserService.prototype.chrome',
                    'debugger.attach',
                    {tabId: },
                    '1.1')
        client.emit('call',
                    'BrowserService.prototype.chrome',
                    'debugger.sendCommand',
                    {tabId: }
                    'Page.navigate',
                    req.body);
                    */
        return;
    }
    if(req.originalUrl.match(/session$/ig)) {
        res.setHeader('Content-Type', 'application/json');
        res.send(JSON.stringify({
            value: {
                sessionId: sessionId,
                capabilities: {}
            }
        }));
        return;
    }
    next();
});
seleniumServer.listen(4444);
module.exports = seleniumServer;
seleniumServer;

